# Compare Area of Interest

Only the area in front of the ego vehicle in the KITTI data is labeled. Because the configuration files for my model are so complicated, and I don't feel confident that I can write up a new one. I want to borrow heavily from the existing one. Whether or not I rotate the ARCS data so that our labels are in the same relative area that the KITTI labels are, I still will need to know where the labeled ARCS area is.

KITTI label area is point_cloud_range = [0, -39.68, -3, 69.12, 39.68, 1]

How to see where the labels are.
Keep track of the highest and lowest values for x, y, and x axis (more precise would be to keep track of highest x + dx/2 and lowest x - dx/2, etc. But I won't worry about that for now. Although. That would make a sort of rectangle that would include areas that are not covered. So maybe it makes sense to convert back to azimuth, height, distance and make a cone that way? 

In [24]:
import os
import numpy as np
from pathlib import Path

In [25]:
# Go through each label file and each label to see where the label area is
DATA_DIR_ROOT = '../data/corrected_labels'

In [26]:
def get_labels(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            dz, dy, dx, x, z, y = map(float, parts[8:14])
            labels.append((dz, dy, dx, x, z, y))
    return labels

In [27]:
def update_extremes(labels, min_max):
    for dz, dy, dx, x, z, y in labels:
        min_x, max_x, min_y, max_y, min_z, max_z = min_max
        # Update min and max values considering the corners of the bounding box
        min_max = [
            min(min_x, x - dx/2), max(max_x, x + dx/2),
            min(min_y, y - dy/2), max(max_y, y + dy/2),
            min(min_z, z - dz/2), max(max_z, z + dz/2)
        ]
    return min_max

In [28]:
def analyze_labeling_area(label_dir):
    min_max = [float('inf'), float('-inf'), float('inf'), float('-inf'), float('inf'), float('-inf')]
    for filename in os.listdir(label_dir):
        if filename.endswith('.txt'):  # Assuming the label files are .txt
            file_path = os.path.join(label_dir, filename)
            labels = get_labels(file_path)
            min_max = update_extremes(labels, min_max)
    return min_max

In [29]:
bounding_box_limits = analyze_labeling_area(DATA_DIR_ROOT)
print("Bounding box limits (min_x, max_x, min_y, max_y, min_z, max_z):", bounding_box_limits)

Bounding box limits (min_x, max_x, min_y, max_y, min_z, max_z): [-51.955, 26.035, -16.25, 39.684999999999995, -4.16, 3.375]


In [42]:
def calculate_center_and_dimensions(min_max):
    min_x, max_x, min_y, max_y, min_z, max_z = min_max
    
    # Calculate the center coordinates
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    center_z = (min_z + max_z) / 2
    
    # Calculate dimensions
    length = max_x - min_x
    width = max_y - min_y
    height = max_z - min_z
    
    return (center_x, center_y, center_z), (length, width, height)

In [44]:
# min_max_values = [0, 69.12, -39.68, 39.68, -3, 1]  # Example values
center, dimensions = calculate_center_and_dimensions(bounding_box_limits)
print("Center (x, y, z):", center)
print("Dimensions (l, w, h):", dimensions)

Center (x, y, z): (-12.959999999999999, 11.717499999999998, -0.39250000000000007)
Dimensions (l, w, h): (77.99, 55.934999999999995, 7.535)
